In [1]:
%display latex

## Métricas invariantes a izquierda en $S_\mathrm{hy}$

In [2]:
structure_constants = {('e1','e2') : {'e2' : 1}, ('e1','e3') : {'e3' : 1}}
hh.<e1,e2,e3> = LieAlgebra(SR, structure_constants) 

In [3]:
hh.is_solvable()

True

In [4]:
hh.is_nilpotent()

False

In [5]:
B = hh.killing_form_matrix()
B

[2 0 0]
[0 0 0]
[0 0 0]

In [6]:
def inner_product_from_matrix(hh, g):
    return lambda x, y: x.to_vector() * g * y.to_vector()

def gram_schmidt(hh, inner_product):
    hh_on_basis = []
    for i, v in enumerate(hh.basis().values()):
        u = v
        for j in range(i):
            u -= inner_product(v, hh_on_basis[j]) * hh_on_basis[j]
        hh_on_basis.append( hh( ((1 / sqrt(inner_product(u, u))) * u).to_vector().canonicalize_radical() ))
    return hh_on_basis

def bracket_contracted(hh, inner_product, on_basis):
    hh_dim = hh.dimension()
    bc = {(i,j,k): None for i in range(hh_dim) for j in range(hh_dim) for k in range(hh_dim)}
    for index in bc.keys():
        i, j, k = index
        bc[index] = inner_product(on_basis[i].bracket(on_basis[j]), on_basis[k])
    return bc

def secc_curv(hh, metric, on_basis, i, j):
    hh_dim = hh.dimension()
    bc = bracket_contracted(hh, metric, on_basis)
    return sum(bc[i,j,k]*(-bc[i,j,k]+bc[j,k,i]+bc[k,i,j])/2 - (bc[i,j,k]-bc[j,k,i]+bc[k,i,j])*(bc[i,j,k]+bc[j,k,i]-bc[k,i,j])/4 - bc[k,i,i]*bc[k,j,j] for k in range(hh_dim))

def Ric_ob(hh, bc, j, h):
    hh_dim = hh.dimension()
    return sum(sum( bc[(i,k,i)] * (bc[(k,j,h)] + bc[(k,h,j)]) / 2 + bc[(i,k,h)] * bc[(i,k,j)] / 4 - bc[(i,j,k)] * bc[(k,h,i)] / 2 + bc[(i,k,i)] * bc[(j,h,k)] / 2 - bc[(i,j,k)] * bc[(i,h,k)] / 2 for i in range(hh_dim)) for k in range(hh_dim))

def Ric_on(hh, g, on_basis):
    hh_dim = hh.dimension()
    ip = inner_product_from_matrix(hh, g)
    on_basis = gram_schmidt(hh, ip)
    bc = bracket_contracted(hh, ip, on_basis)
    return matrix(hh_dim, hh_dim, lambda i, j: Ric_ob(hh, bc, i, j))

In [7]:
I3 = identity_matrix(SR,3)
metric = inner_product_from_matrix(hh, I3)

In [8]:
on_basis = gram_schmidt(hh, metric)
on_basis

[e1, e2, e3]

In [9]:
# Einstein
Ric_on(hh, I3, on_basis)

[-2  0  0]
[ 0 -2  0]
[ 0  0 -2]

In [10]:
# Curvaturas seccionales en la base ortonormal
for i in range(3):
    for j in range(i+1,3):
        display(secc_curv(hh,metric, on_basis, i, j))

-1

-1

-1

## Todas las métricas en $S_\mathrm{hy}$ tienen curvatura seccional constante negativa

In [11]:
r1, r2, r3 = var("r1 r2 r3")
g = diagonal_matrix(SR,[r1, r2, r3])
display(g)
metric = inner_product_from_matrix(hh, g)

[r1  0  0]
[ 0 r2  0]
[ 0  0 r3]

In [12]:
on_basis = gram_schmidt(hh, metric)
on_basis

[1/sqrt(r1)*e1, 1/sqrt(r2)*e2, 1/sqrt(r3)*e3]

In [13]:
Ric_on(hh, g, on_basis)

[-2/r1     0     0]
[    0 -2/r1     0]
[    0     0 -2/r1]

In [14]:
# Curvaturas seccionales en la base ortonormal
for i in range(3):
    for j in range(i+1,3):
        display(secc_curv(hh,metric, on_basis, i, j))

-1/r1

-1/r1

-1/r1

## Ejemplos con $\dim > 3$

In [15]:
structure_constants = {('e1','e2') : {'e2' : 1}, ('e1','e3') : {'e3' : 1},
                       ('e1','e4') : {'e4' : 1}, ('e1','e5') : {'e5' : 1},
                      }
hh.<e1,e2,e3,e4,e5> = LieAlgebra(SR, structure_constants) 

In [16]:
hh.is_solvable()

True

In [17]:
hh.is_nilpotent()

False

In [18]:
B = hh.killing_form_matrix()
B

[4 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]

In [19]:
a1, a2, a3, a4, a5, a6 = var("a1 a2 a3 a4 a5 a6")
g = matrix(SR,[[a1, a2, 0, 0, 0],
               [a2, a3, 0, 0, 0], 
               [0, 0, a4, 0, 0],
               [0, 0, 0, a5, 0],
               [0, 0, 0, 0, a6]]
          )
display(g)
metric = inner_product_from_matrix(hh, g)

[a1 a2  0  0  0]
[a2 a3  0  0  0]
[ 0  0 a4  0  0]
[ 0  0  0 a5  0]
[ 0  0  0  0 a6]

In [20]:
g.det().factor()

-(a2^2 - a1*a3)*a4*a5*a6

In [21]:
on_basis = gram_schmidt(hh, metric)
on_basis

[1/sqrt(a1)*e1,
 (-a2/(sqrt(-a2^2+a1*a3)*sqrt(a1)))*e1 + (sqrt(a1)/sqrt(-a2^2+a1*a3))*e2,
 1/sqrt(a4)*e3,
 1/sqrt(a5)*e4,
 1/sqrt(a6)*e5]

In [22]:
# Einstein?
Ric_on(hh, g, on_basis)

[                                                                    -(a2^2/((a2^2 - a1*a3)*sqrt(a1)) - sqrt(a1)*a3/(a2^2 - a1*a3))^2 + 4*a2^2/((a2^2 - a1*a3)*a1) - 3/a1                                                                                                                                                                        0                                                                                                                                                                        0                                                                                                                                                                        0                                                                                                                                                                        0]
[                                      -3*a2*(a2^2/((a2^2 - a1*a3)*sqrt(a1)) - sqrt(a1)*a3/(a2^2 - a1*a3))/(sqrt(-a2^2 + a1*a3)*sqrt(a1)) + 3*a2/(sqrt(-a2^2 + a1*a3)*a1) -(a2^2/((a2^2 - a1*a3)*sqrt(a1)) - sqrt(a1)*a3/(a2^2 - a1*a3))^2 - 3*(a2^2/((a2^2 - a1*a3)*sqrt(a1)) - sqrt(a1)*a3/(a2^2 - a1*a3))/sqrt(a1) + 4*a2^2/((a2^2 - a1*a3)*a1)                                                                                                                                                                        0                                                                                                                                                                        0                                                                                                                                                                        0]
[                                                                                                                                                                       0                                                                                                                                                                        0                                                              -(a2^2/((a2^2 - a1*a3)*sqrt(a1)) - sqrt(a1)*a3/(a2^2 - a1*a3))/sqrt(a1) + 4*a2^2/((a2^2 - a1*a3)*a1) - 3/a1                                                                                                                                                                        0                                                                                                                                                                        0]
[                                                                                                                                                                       0                                                                                                                                                                        0                                                                                                                                                                        0                                                              -(a2^2/((a2^2 - a1*a3)*sqrt(a1)) - sqrt(a1)*a3/(a2^2 - a1*a3))/sqrt(a1) + 4*a2^2/((a2^2 - a1*a3)*a1) - 3/a1                                                                                                                                                                        0]
[                                                                                                                                                                       0                                                                                                                                                                        0                                                                                                                                                                        0                                                                                                         

In [23]:
# Einstein!
Ric_on(hh, g, on_basis).factor()

[4*a3/(a2^2 - a1*a3)                   0                   0                   0                   0]
[                  0 4*a3/(a2^2 - a1*a3)                   0                   0                   0]
[                  0                   0 4*a3/(a2^2 - a1*a3)                   0                   0]
[                  0                   0                   0 4*a3/(a2^2 - a1*a3)                   0]
[                  0                   0                   0                   0 4*a3/(a2^2 - a1*a3)]

In [24]:
# Curvaturas seccionales en la base ortonormal
for i in range(5):
    for j in range(i+1,5):
        display(secc_curv(hh,metric, on_basis, i, j).factor())
        print("")

a3/(a2^2 - a1*a3)

a3/(a2^2 - a1*a3)

a3/(a2^2 - a1*a3)

a3/(a2^2 - a1*a3)

a3/(a2^2 - a1*a3)

a3/(a2^2 - a1*a3)

a3/(a2^2 - a1*a3)

a3/(a2^2 - a1*a3)

a3/(a2^2 - a1*a3)

a3/(a2^2 - a1*a3)